## Задача 1

Обучить любую модель классификации на датасете IRIS до применения PCA и после него. Сравнить качество классификации по отложенной выборке.

In [3]:
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
import random


from matplotlib.colors import ListedColormap
from sklearn import datasets
from sklearn import model_selection
from sklearn import datasets

import numpy as np

from pprint import pprint

In [4]:
"""
Полжим сюда семечко 
"""
random.seed(123)

In [5]:
"""
Загружаем датасет ирис 
"""
iris = datasets.load_iris()

In [6]:
X = iris.data
y = iris.target

In [40]:
train_data, test_data, train_labels, test_labels = model_selection.train_test_split(X, y,
                                                                                     test_size = 0.3,
                                                                                     random_state = 42)

In [41]:
# Реализуем класс узла

class Node:
    
    def __init__(self, index, t, true_branch, false_branch):
        self.index = index  # индекс признака, по которому ведется сравнение с порогом в этом узле
        self.t = t  # значение порога
        self.true_branch = true_branch  # поддерево, удовлетворяющее условию в узле
        self.false_branch = false_branch  # поддерево, не удовлетворяющее условию в узле
        
    def get_index(self):
        return self.index

In [42]:
# И класс терминального узла (листа)

class Leaf:
    
    def __init__(self, data, labels, index):
        self.index = index
        self.data = data
        self.labels = labels
        self.prediction = self.predict()
        
    def predict(self):
        # подсчет количества объектов разных классов
        classes = {}  # сформируем словарь "класс: количество объектов"
        for label in self.labels:
            if label not in classes:
                classes[label] = 0
            classes[label] += 1
        #  найдем класс, количество объектов которого будет максимальным в этом листе и вернем его    
        prediction = max(classes, key=classes.get)
        return prediction   

In [43]:
class Criteriy:
    # Расчет критерия Джини
    def __init__(self, labels):
        self.labels = labels
        
    def count_gini(labels):
        #  подсчет количества объектов разных классов
        classes = {}
        for label in labels:
            if label not in classes:
                classes[label] = 0
            classes[label] += 1

        #  расчет критерия
        impurity = 1
        for label in classes:
            p = classes[label] / len(labels)
            impurity -= p ** 2

        return impurity
    
    # Расчет качества

    def count_quality(left_labels, right_labels, current_gini):

        # доля выбоки, ушедшая в левое поддерево
        p = float(left_labels.shape[0]) / (left_labels.shape[0] + right_labels.shape[0])

        return current_gini - p *Criteriy.count_gini(left_labels) - (1 - p) * Criteriy.count_gini(right_labels)

In [44]:
class Split:
    
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
        
    def split(data, labels, index, t):
    
        left = np.where(data[:, index] <= t)
        right = np.where(data[:, index] > t)

        true_data = data[left]
        false_data = data[right]
        true_labels = labels[left]
        false_labels = labels[right]

        return true_data, false_data, true_labels, false_labels
    
    # Нахождение наилучшего разбиения

    def find_best_split(data, labels):

        #  обозначим минимальное количество объектов в узле
        min_leaf = 5

        current_gini = Criteriy.count_gini(labels)

        best_quality = 0
        best_t = None
        best_index = None

        n_features = data.shape[1]

        # выбор индекса из подвыборки длиной sqrt(n_features)
        subsample = Oob.get_subsample(n_features)

        for index in subsample:
            t_values = [row[index] for row in data]

            for t in t_values:
                true_data, false_data, true_labels, false_labels = Split.split(data, labels, index, t)
                #  пропускаем разбиения, в которых в узле остается менее 5 объектов
                if len(true_data) < min_leaf or len(false_data) < min_leaf:
                    continue

                current_quality = Criteriy.count_quality(true_labels, false_labels, current_gini)

                #  выбираем порог, на котором получается максимальный прирост качества
                if current_quality > best_quality:
                    best_quality, best_t, best_index = current_quality, t, index

        return best_quality, best_t, best_index

In [45]:
class Forest:
    
    def __init__(self, data, labels, n_trees):
        self.data = data
        self.labels = labels
        self.n_trees = n_trees
        self.forest = self.random_forest(self.data, self.labels, self.n_trees)
    
    # Построение дерева с помощью рекурсивной функции

    def build_tree(self, data, labels):

        quality, t, index = Split.find_best_split(data, labels)

        #  Базовый случай - прекращаем рекурсию, когда нет прироста в качества
        if quality == 0:
            return Leaf(data, labels, index)

        true_data, false_data, true_labels, false_labels = Split.split(data, labels, index, t)

        # Рекурсивно строим два поддерева
        true_branch = self.build_tree(true_data, true_labels)
        false_branch = self.build_tree(false_data, false_labels)

        # Возвращаем класс узла со всеми поддеревьями, то есть целого дерева
        return Node(index, t, true_branch, false_branch)
    
    def random_forest(self, data, labels, n_trees):
        forest = []
        bootstrap, out_of_bag = Oob.get_bootstrap(data, labels, n_trees)

        for b_data, b_labels in bootstrap:
            forest.append(self.build_tree(b_data, b_labels))

        return forest, out_of_bag

In [46]:
class Classification:
    
    def __init__(self, forest, data):
        self.forest = forest
        self.data = data
        self.tree_vote = self.tree_vote(self.forest, self.data)
    
    # Функция классификации отдельного объекта

    def classify_object(self, obj, node):

        #  Останавливаем рекурсию, если достигли листа
        if isinstance(node, Leaf):
            answer = node.prediction
            return answer
        
        if obj[node.index] <= node.t:
            return self.classify_object(obj, node.true_branch)
        else:
            return self.classify_object(obj, node.false_branch)
        
    # функция формирования предсказания по выборке на одном дереве

    def predict(self, data, tree):

        classes = []
        for obj in data:
            prediction = self.classify_object(obj, tree)
            classes.append(prediction)
        return classes
    
    # предсказание голосованием деревьев

    def tree_vote(self, forest, data):

        # добавим предсказания всех деревьев в список
        predictions = []
        for tree in forest:
            predictions.append(self.predict(data, tree))

        # сформируем список с предсказаниями для каждого объекта
        predictions_per_object = list(zip(*predictions))

        # выберем в качестве итогового предсказания для каждого объекта то,
        # за которое проголосовало большинство деревьев
        voted_predictions = []
        for obj in predictions_per_object:
            voted_predictions.append(max(set(obj), key=obj.count))

        return voted_predictions

In [47]:
class Oob:

    def __init__(self, forest, out_of_bag, data, labels):
        self.forest = forest
        self.out_of_bag = out_of_bag
        self.data = data
        self.labels = labels
        self.count = self.oob(self.forest, self.out_of_bag, self.data, self.labels)

    # Расчет метрики OOB
    def oob(self, forest, out_of_bag, data, labels):
        predictions = []
        actual = []
        for i in range(data.shape[0]):
            index_tree = []
            for idx, tree in enumerate(out_of_bag):
                if i not in tree:
                    index_tree.append(idx)
            if len(index_tree) != 0:
                new_forest = [forest[k] for k in index_tree]
                predictions.append(Classification(new_forest, [data[i]]).tree_vote[0])
                actual.append(labels[i])
        
        return Accuracy(actual, predictions).meric

    
    def get_bootstrap(data, labels, N) -> (list, list):
        n_samples = data.shape[0]
        bootstrap = []
        indexes = []
        
        for i in range(N):
            sample_indexes = []
            b_data = np.zeros(data.shape)
            b_labels = np.zeros(labels.shape)
            
            for j in range(n_samples):
                sample_index= random.randint(0, n_samples -1)
                b_data[j] = data[sample_index]
                b_labels[j] = labels[sample_index]
                sample_indexes.append(sample_index)
            
            indexes.append(sample_indexes)
            bootstrap.append((b_data, b_labels))
        
        return bootstrap, indexes
        
        
    def get_subsample(len_sample):
        # будем сохранять не сами признаки, а их индексы
        sample_indexes = [i for i in range(len_sample)]

        len_subsample = int(np.sqrt(len_sample))
        subsample = []

        random.shuffle(sample_indexes)
        subsample = sample_indexes[0:len_subsample].copy()
    #     for _ in range(len_subsample):
    #         subsample.append(sample_indexes.pop())

        return subsample

In [48]:
class Accuracy:
    
    def __init__(self, actual, predicted):
        self.actual = actual
        self.predicted = predicted
        self.meric = self.accuracy_metric(self.actual, self.predicted)
        
        
    def accuracy_metric(self, actual, predicted):
        correct = 0
        for i in range(len(actual)):
            if actual[i] == predicted[i]:
                correct += 1
        return correct / float(len(actual)) * 100.0

In [49]:
n_trees = 10

Лес трейн

In [50]:
forest = Forest(train_data, train_labels, n_trees)
train_answers = Classification(forest.forest[0], train_data)

In [51]:
train_accuracy = Accuracy(train_answers.tree_vote, train_labels)

In [52]:
print(train_accuracy.meric)

96.19047619047619


Лес тест

In [53]:
test_answers = Classification(forest.forest[0], test_data)
test_accuracy = Accuracy(test_answers.tree_vote, test_labels)

In [54]:
test_accuracy.meric

100.0

Лес oob

In [55]:
train_oob = Oob(forest.forest[0], forest.forest[1], train_data, train_labels)

In [56]:
train_oob.count

93.26923076923077

### Реализуем метод PCA

In [7]:
# Для начала отмасштабируем выборку
X_ = X.astype(float)

rows, cols = X_.shape

# центрирование - вычитание из каждого значения среднего по строке
means = X_.mean(0)
for i in range(rows):
    for j in range(cols):
        X_[i, j] -= means[j]

# деление каждого значения на стандартное отклонение
std = np.std(X_, axis=0)
for i in range(cols):
    for j in range(rows):
        X_[j][i] /= std[i]

In [58]:
# Найдем собственные векторы и собственные значения (англ. Eigenvalues)
 
covariance_matrix = X_.T.dot(X_)

eig_values, eig_vectors = np.linalg.eig(covariance_matrix)

# сформируем список кортежей (собственное значение, собственный вектор)
eig_pairs = [(np.abs(eig_values[i]), eig_vectors[:, i]) for i in range(len(eig_values))]

# и отсортируем список по убыванию собственных значений
eig_pairs.sort(key=lambda x: x[0], reverse=True)

print('Собственные значения в порядке убывания:')
for i in eig_pairs:
    print(i)

Собственные значения в порядке убывания:
(437.77467247979905, array([ 0.52106591, -0.26934744,  0.5804131 ,  0.56485654]))
(137.10457072021032, array([-0.37741762, -0.92329566, -0.02449161, -0.06694199]))
(22.01353133569723, array([-0.71956635,  0.24438178,  0.14212637,  0.63427274]))
(3.107225464292853, array([ 0.26128628, -0.12350962, -0.80144925,  0.52359713]))


Оценим долю дисперсии, которая описывается найденными компонентами.


In [59]:
eig_sum = sum(eig_values)
var_exp = [(i / eig_sum) * 100 for i in sorted(eig_values, reverse=True)]
cum_var_exp = np.cumsum(var_exp)
print(f'Доля дисперсии, описываемая каждой из компонент \n{var_exp}')

# а теперя оценим кумулятивную (то есть накапливаемую) дисперсию при учитывании каждой из компонент
print(f'Кумулятивная доля дисперсии по компонентам \n{cum_var_exp}')

Доля дисперсии, описываемая каждой из компонент 
[72.96244541329992, 22.850761786701742, 3.668921889282875, 0.517870910715476]
Кумулятивная доля дисперсии по компонентам 
[ 72.96244541  95.8132072   99.48212909 100.        ]


Таким образом, первая главная компонента описывает почти 73% информации, а первые две в сумме - 95.8%. В то же время последняя компонента описывает всего 0.5% и может быть отброжена без страха значительных потерь в качестве нашего анализа. Мы отбросим последние две компоненты, оставив первые две.

In [60]:
# Сформируем вектор весов из собственных векторов, соответствующих первым двум главным компонентам
W = np.hstack((eig_pairs[0][1].reshape(4,1), eig_pairs[1][1].reshape(4,1)))

print(f'Матрица весов W:\n', W)

Матрица весов W:
 [[ 0.52106591 -0.37741762]
 [-0.26934744 -0.92329566]
 [ 0.5804131  -0.02449161]
 [ 0.56485654 -0.06694199]]


In [61]:
# Сформируем новую матрицу "объекты-признаки"
Z = X_.dot(W)

In [62]:
Z[:10]

array([[-2.26470281, -0.4800266 ],
       [-2.08096115,  0.67413356],
       [-2.36422905,  0.34190802],
       [-2.29938422,  0.59739451],
       [-2.38984217, -0.64683538],
       [-2.07563095, -1.48917752],
       [-2.44402884, -0.0476442 ],
       [-2.23284716, -0.22314807],
       [-2.33464048,  1.11532768],
       [-2.18432817,  0.46901356]])

In [63]:
train_data, test_data, train_labels, test_labels = model_selection.train_test_split(Z, y,
                                                                                     test_size = 0.3,
                                                                                     random_state = 42)

In [64]:
forest = Forest(train_data, train_labels, n_trees)
train_answers = Classification(forest.forest[0], train_data)

In [65]:
train_accuracy = Accuracy(train_answers.tree_vote, train_labels)
print(train_accuracy.meric)

95.23809523809523


In [66]:
test_answers = Classification(forest.forest[0], test_data)
test_accuracy = Accuracy(test_answers.tree_vote, test_labels)

In [67]:
print(test_accuracy.meric)

88.88888888888889


Видим, что точность упала

# Задача 2
2*. Написать свою реализацию метода главных компонент с помощью сингулярного разложения с использованием функции numpy.linalg.svd()

In [8]:
def SVD(X, n):
    u, s, vh = np.linalg.svd(X, full_matrices=True)
    num = min(n, len(s))
    W = vh.transpose()[:, :num]
    return np.dot(X, W)

In [9]:
Z = SVD(X_, 2)
Z[:10]

array([[-2.26470281, -0.4800266 ],
       [-2.08096115,  0.67413356],
       [-2.36422905,  0.34190802],
       [-2.29938422,  0.59739451],
       [-2.38984217, -0.64683538],
       [-2.07563095, -1.48917752],
       [-2.44402884, -0.0476442 ],
       [-2.23284716, -0.22314807],
       [-2.33464048,  1.11532768],
       [-2.18432817,  0.46901356]])